<a href="https://colab.research.google.com/github/sutanto5/Amazon_WebScraper/blob/main/Amazon_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#This model will be used to run sentiment analysis on my book review data


In [ ]:
me

# **NLTK Basics**

In [38]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [39]:
#import data
df = pd.read_csv('hillbilly_elegy.csv')
df.head()

,User Name,Review Date,Review Body
0,PRNLM,"Reviewed in the United States on July 25, 2016","There is a lot to take in here, even for someo..."
1,J. Foster,"Reviewed in the United States on August 12, 2017","J.D. Vance chronicles his ""upwardly mobile"" jo..."
2,Ted Lehmann,"Reviewed in the United States on July 13, 2017",Hillbilly Elegy: A Memoir of a Family and Cult...
3,Fnu Lnu,"Reviewed in the United States on December 9, 2016","This is an exceptional memoir, particularly fr..."
4,Richard B. Schwartz,"Reviewed in the United States on July 3, 2017",It may be that the most interesting thing abou...


In [40]:
df['Review Body'].values[2]

"Hillbilly Elegy: A Memoir of a Family and Culture in Crisis by J.D. Vance (Harper Collins, 2016, $12.59/15.99) has been on the NY Times Best Seller List for forty-nine weeks, and at the time of this writing, stands at number two. This touching, revealing, warm, sad, and inspiring memoir, written by a Yale Law School graduate whose childhood was spent in the hills of eastern Kentucky and the migrated community of Middletown, OH, opens many sores while explaining in the most human and personal terms possible the pain and misunderstanding that harms working class and poor white Americans in the Hheartlands. Throughout this tale Vance sometimes mentions findings of academic studies and other research, using them to support or introduce his own poignant experiences, but, most of all, this is the story of one man's ability to persevere in an environment where success such as he has experienced is rare, and , according to him psychically costly as well as economically expensive.\n\nLiving wi

In [41]:
df.shape

(100, 3)

# **Basic NLTK**

In [42]:
example = df['Review Body'].values[20]
example

'This in some ways mirrors my own life story. I was in three homes and an orphanage. I know many of these stories and have lived the flight syndrome all my life.\n\nThank you sir for your vulnerability and willingness to deal with your life hurts.\n\nBlessings and my prayers and support are with you Usha.\n\nYour a great man JD Vance!'

In [44]:
tokens = nltk.word_tokenize(example)
tokens[:10]

['This', 'in', 'some', 'ways', 'mirrors', 'my', 'own', 'life', 'story', '.']

In [48]:
#part of speech tagging
nltk.download('averaged_perceptron_tagger_eng')
tagged = nltk.pos_tag(tokens)
tagged[:10]

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


[('This', 'DT'),
 ('in', 'IN'),
 ('some', 'DT'),
 ('ways', 'NNS'),
 ('mirrors', 'NNS'),
 ('my', 'PRP$'),
 ('own', 'JJ'),
 ('life', 'NN'),
 ('story', 'NN'),
 ('.', '.')]

In [55]:
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


(S
  This/DT
  in/IN
  some/DT
  ways/NNS
  mirrors/NNS
  my/PRP$
  own/JJ
  life/NN
  story/NN
  ./.
  I/PRP
  was/VBD
  in/IN
  three/CD
  homes/NNS
  and/CC
  an/DT
  orphanage/NN
  ./.
  I/PRP
  know/VBP
  many/JJ
  of/IN
  these/DT
  stories/NNS
  and/CC
  have/VBP
  lived/VBN
  the/DT
  flight/NN
  syndrome/NN
  all/DT
  my/PRP$
  life/NN
  ./.
  Thank/NNP
  you/PRP
  sir/VBP
  for/IN
  your/PRP$
  vulnerability/NN
  and/CC
  willingness/NN
  to/TO
  deal/VB
  with/IN
  your/PRP$
  life/NN
  hurts/NNS
  ./.
  Blessings/NNS
  and/CC
  my/PRP$
  prayers/NNS
  and/CC
  support/NN
  are/VBP
  with/IN
  you/PRP
  Usha/VBP
  ./.
  Your/PRP$
  a/DT
  great/JJ
  man/NN
  JD/NNP
  Vance/NNP
  !/.)


# **VADER Sentiment Scoring**


*   Bag of words approach


    1.   Stop words are removed
    2.   Each word is scored and combined for a total score



In [58]:
#imports
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Practice SIA

In [59]:
sia = SentimentIntensityAnalyzer()

In [60]:
#compound is a combination of negative and positive -> from -1 to 1
sia.polarity_scores('I am so happy') #prints positivity

{'neg': 0.0, 'neu': 0.334, 'pos': 0.666, 'compound': 0.6115}

In [61]:
sia.polarity_scores('I am so sad') #prints negativity

{'neg': 0.629, 'neu': 0.371, 'pos': 0.0, 'compound': -0.5256}

In [62]:
sia.polarity_scores(example)

{'neg': 0.069, 'neu': 0.72, 'pos': 0.21, 'compound': 0.8805}

In [71]:
res = {}

for _, row in tqdm(df.iterrows(), total=len(df)):
    review = row['Review Body']  # Ensure column name is correct
    user_id = row['User Name']   # Ensure column name is correct

    if pd.isna(review):  # Skip NaN reviews
        continue

    res[user_id] = sia.polarity_scores(review)  # Convert to string in case of unexpected NaNs

  0%|          | 0/100 [00:00<?, ?it/s]

In [72]:
#dict
res

{'PRNLM': {'neg': 0.059, 'neu': 0.81, 'pos': 0.131, 'compound': 0.9929},
 'J. Foster': {'neg': 0.107, 'neu': 0.798, 'pos': 0.095, 'compound': -0.8432},
 'Ted Lehmann': {'neg': 0.113, 'neu': 0.743, 'pos': 0.144, 'compound': 0.9888},
 'Fnu Lnu': {'neg': 0.048, 'neu': 0.723, 'pos': 0.229, 'compound': 0.9997},
 'Richard B. Schwartz': {'neg': 0.022,
  'neu': 0.822,
  'pos': 0.156,
  'compound': 0.9989},
 'Tom and Kristen Wannemacher': {'neg': 0.054,
  'neu': 0.787,
  'pos': 0.159,
  'compound': 0.9654},
 'Robert A Haworth': {'neg': 0.099,
  'neu': 0.698,
  'pos': 0.203,
  'compound': 0.9992},
 'Dawn': {'neg': 0.072, 'neu': 0.789, 'pos': 0.139, 'compound': 0.8993},
 'nicole w brown': {'neg': 0.102,
  'neu': 0.809,
  'pos': 0.089,
  'compound': -0.9816},
 'Liza': {'neg': 0.07, 'neu': 0.725, 'pos': 0.205, 'compound': 0.9857},
 'ironhorseranch': {'neg': 0.057,
  'neu': 0.744,
  'pos': 0.199,
  'compound': 0.9713},
 'Chris': {'neg': 0.105, 'neu': 0.781, 'pos': 0.114, 'compound': 0.4433},
 'Beard

In [78]:
#T transpsoes -> puts res in dataframe
pd.DataFrame(res).T

,neg,neu,pos,compound
PRNLM,0.059,0.810,0.131,0.9929
J. Foster,0.107,0.798,0.095,-0.8432
Ted Lehmann,0.113,0.743,0.144,0.9888
Fnu Lnu,0.048,0.723,0.229,0.9997
Richard B. Schwartz,0.022,0.822,0.156,0.9989
...,...,...,...,...
Russ Sindelar,0.000,0.717,0.283,0.8516
Sharon G.,0.034,0.626,0.340,0.9846
Wendy Oconnell,0.103,0.708,0.189,0.9979
Leroy F. Gayle,0.086,0.525,0.389,0.8994
